# Threshold de Classificação

# Setup

OBS: Instalar a lib yellowbrick

In [ ]:
#descomente abaixo para instalar
#!pip install yellowbrick

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.metrics import auc, precision_recall_curve, log_loss,confusion_matrix, recall_score, precision_score
from sklearn.compose import ColumnTransformer
from yellowbrick.classifier import DiscriminationThreshold
import warnings
warnings.filterwarnings('ignore')


# Dados

Realizem a divisão dos dados em Treino e Teste, utilizando o parâmetro ```random_state``` = 123

## Divisão dos Dados

Fazendo seleção não estratificada

Fazendo seleção estratificada

Dividindo em treino e teste de forma estratificada

Divida seus dados de forma estratificada ```random_state=123``` e separe as colunas categóricas das numéricas em duas listas.

# Pipelines

Crie um pipeline para processamento dos dados de entrada. Primeiro crie um pipeline categórico com ```OneHotEncoder``` e depois um pipeline numérico com ```MinMaxScaler```

Combine tudo com o ```ColumnTransformer```

# Modelagem

Compare os seguintes algoritmos, utilizando a métrica log loss.:
- Regressão Logística
- SVM
- KNN
- Decision Tree
- Random Forest

Construa um pipeline para cada um, utilizando o ```ColumnTransformer``` do passo anterior.

Vamos rodar todos os modelos e checar a performance com a log_loss. Dica, crie uma lista de modelos e rode um for loop.

# Avaliação do Threshold

Treine o modelo campeão.

Avalie as métricas em relação ao threshold escolhido. Dica: use a biblioteca ```yellowbrick```

Com esse visualizador temos, além da mediana, uma banda que representa 90% de confiança

Como decidir o threshold? 

Isso vai depender da definição de sucesso escolhida lá no início do projeto!!

Se a ideia é aumentar a conversão, sem perder volume, devemos olhar pra pra queue rate e para precision!.

Vamos fazer um exercício para definir o threshold. 

Em tese, destes 1000, 145 irão desativar o produto da empresa. Devido ao coronavírus, a empresa decidiu reduzir o número de pessoas no customer service. Atualmente, 7 pessoas trabalham lá e em um mês, elas fazem em média 100 ligações. A taxa atual de precisão é 15%, ou seja, das 700 ligações por mês (considerando 1 ligação por cliente), eles conseguem recuperar 105 pessoas. A empresa pensa em demitir 2 pessoas e para isso, pediu que você desenvolvesse um modelo de churn para auxiliar a operação de customer service. 

Baseado nos resultados que tivemos, é possível demitir 2 pessoas, mantendo a mesma precisão e recuperando no mínimo o mesmo de antes? 

Podemos ver esse problema de uma outra forma. E se em vez de responder a esse tipo de pergunta, quiséssemos escolher o threshold que permitisse que a empresa lucrasse mais?

Crie uma função de custo personalizada para o problema, levando em conta falsos positivos, falsos negativos, verdadeiros positivos e verdadeiros negativos e escolha o threshold que maximiza ou minimiza essa função (dependendo do problema). Qual o impacto financeiro de um Falso Positivo? E de um Falso Negativo? E de um Verdadeiro Positivo? E de um Verdadeiro Negativo?

Para esse exercício, precisamos definir os custos e receita de cada componente da matriz de confusão.

**Verdadeiro Positivo:** O verdadeiro positivo é o cara que iria desativar o produto e foi identificado corretamente pelo modelo. Esse cara gasta em média R$ 150 por mês com o produto da empresa e tem uma probabilidade de 15 por cento de ser retido, caso liguem para ele. 

 

Dessa forma podemos criar uma expressão que traz em média o "custo" de um Verdadeiro Positivo. Lembrando que a ligação também tem um custo, que vamos admitir que seja de R$ 10,00. Dessa forma, a expressão para o Verdadeiro positivo fica:    
     VP = 0.15*150 - 10



**Falso Positivo**: O falso positivo insere o custo da ligação. Dessa forma  a expressão fica:
FP = 150 - 10

**Verdadeiro Negativo**: O verdadeiro negativo traz lucro para a empresa, pois consome em média 150 reais.<br>
VN = 150

**Falso Negativo**: Por fim, o Falso Negativo é o cara que simplesmente deixa o produto e não transaciona mais, vamos admitir que é uma perda para a empresa. Como cada cliente transaciona em média 150 reais, a empresa terá uma perda equivalente.
<br>
FN = -150